# Automated COVID-19 Monitoring ETL Project
**Author**: Moch Nabil Farras Dhiya

**E-mail**: nabilfarras923@gmail.com

**Institution**: Bandung Institute of Technology

**Student ID**: 10120034


---

**About**: This is a side-project created in order to monitor COVID-19 data from WHO data source by the following steps:


1.   Extracting data from WHO Website
2.   Transform the data (making sure it is usable and consistent)
3.   Upload the final data to Local MySQL Database
4.   Visualize the data for end-users by Tableau

These steps will be automatically implemented on daily basis, so we only need to monitor the dashboard.

**NOTE**: In this project, I use on-site database in order to minimize the cost as Cloud Databases are mostly paid.

# Import Modules

In [1]:
!pip install requests lxml html5lib beautifulsoup4 sqlalchemy pymysql pycountry

In [2]:
# Connect to local
import os

# Extracting file from URL (WHO Data) and scraping data from another websource (Worldometers Data)
import requests
# from bs4 import BeautifulSoup

# Importing and transforming file
import pandas as pd

# Data manipulation
import numpy as np
import re # Cleaning texts
import datetime as dt # Datetime manipulation
import pycountry # List of countries

# Connect Colab with Local MySQL Database and delete existing table
import sqlalchemy as sql
from sqlalchemy.ext.declarative import declarative_base

# Notify is there is an error to email
import logging
import logging.handlers

# Connect to MySQL Local Database

In [3]:
print("================================")
print("Connecting to Local DB .....")

# Credentials to database connection
hostname = "localhost"
dbname = "covid_19"
uname = "root"
pwd = "Kazekage1234"

# Create SQLAlchemy engine to connect to MySQL Database
sqlEngine = sql.create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}" \
				.format(host = hostname,
                db = dbname,
                user = uname,
                pw = pwd))

dbConnection    = sqlEngine.connect()

print("Successfully to Local DB.")
print("================================")
print()

Connecting to Local DB .....
Successfully to Local DB.



# Extract WHO File (CSV)

## Daily Case and Death

In [4]:
save_path = 'D:\Kuliah\Project\Python\Automated Covid-19 Monitoring'

In [5]:
# Get content from URL
daily_case_death_url = 'https://covid19.who.int/WHO-COVID-19-global-data.csv'
req = requests.get(daily_case_death_url)
url_content = req.content

file_name = 'daily_case_death.csv'
completeName = os.path.join(save_path, file_name)
csv_file_0 = open(completeName, 'wb') # Save to directed folder

csv_file_0.write(url_content)
csv_file_0.close()

## Latest Case and Death

In [6]:
# Get content from URL
latest_case_death_url = 'https://covid19.who.int/WHO-COVID-19-global-table-data.csv'
req = requests.get(latest_case_death_url)
url_content = req.content

file_name = 'latest_case_death.csv'
completeName = os.path.join(save_path, file_name)
csv_file_1 = open(completeName, 'wb') # Save to directed folder

csv_file_1.write(url_content)
csv_file_1.close()

## Vaccination Data

In [7]:
# Get content from URL
vaccine_url = 'https://covid19.who.int/who-data/vaccination-data.csv'
req = requests.get(vaccine_url)
url_content = req.content

file_name = 'vaccination.csv'
completeName = os.path.join(save_path, file_name)
csv_file_2 = open(completeName, 'wb') # Save to directed folder

csv_file_2.write(url_content)
csv_file_2.close()

## Vaccine Metadata

In [8]:
# Get content from URL
vaccine_md_url = 'https://covid19.who.int/who-data/vaccination-metadata.csv'
req = requests.get(vaccine_md_url)
url_content = req.content

file_name = 'vaccination_md.csv'
completeName = os.path.join(save_path, file_name)
csv_file_3 = open(completeName, 'wb') # Save to directed folder

csv_file_3.write(url_content)
csv_file_3.close()

# Scrape External Data

In [9]:
# url_main = "https://www.worldometers.info/coronavirus/#main_table"
# url_weekly = "https://www.worldometers.info/coronavirus/weekly-trends/#weekly_table"

# r_main = requests.get(url_main)
# df_main_list = pd.read_html(r_main.text) # this parses all the tables in webpages to a list
# df_main = df_main_list[0]

# r_weekly = requests.get(url_weekly)
# df_weekly_list = pd.read_html(r_weekly.text) # this parses all the tables in webpages to a list
# df_weekly = df_weekly_list[0]

# Import Data

In [10]:
print("================================")
print("Importing Data .....")
print()

# Today's Daily Case and Death Data
now_df_daily_cd = pd.read_csv('daily_case_death.csv', index_col = False)

# Today's Latest Case and Death Data
now_df_latest_cd = pd.read_csv('latest_case_death.csv', index_col = False)

# Today's Vaccination Data
now_df_vaccination = pd.read_csv('vaccination.csv', index_col = False) 

# Today's Vaccination Metadata Data
now_df_vaccination_md = pd.read_csv('vaccination_md.csv', index_col = False) 

print("Data have been imported")
print()

Importing Data .....

Data have been imported



In [11]:
print("Searching for another Data in the Local DB .....")
cond = False # Assuming there is no data in database yet
try:    
    # Yesterday's Daily Case and Death Data
    yesterday_df_daily_cd = pd.read_sql('SELECT * FROM covid_19.daily_case_death', dbConnection);

    # Yesterday's Latest Case and Death Data
    yesterday_df_latest_cd = pd.read_sql('SELECT * FROM covid_19.latest_case_death', dbConnection);

    # Yesterday's Vaccination Data
    yesterday_df_vaccination = pd.read_sql('SELECT * FROM covid_19.daily_vaccination', dbConnection);

    # Yesterday's Vaccination Metadata Data
    yesterday_df_vaccination_md = pd.read_sql('SELECT * FROM covid_19.vaccination_md', dbConnection);
    
    cond = True
    print("Data found.")

except Exception as e:
    print(e)
    print("No Data in Local DB yet.")

print("================================")
print()

Searching for another Data in the Local DB .....
Data found.



# Initial EDA

## Daily Case and Death

In [12]:
now_df_daily_cd

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
0,2020-01-03,AF,Afghanistan,EMRO,0,0,0,0
1,2020-01-04,AF,Afghanistan,EMRO,0,0,0,0
2,2020-01-05,AF,Afghanistan,EMRO,0,0,0,0
3,2020-01-06,AF,Afghanistan,EMRO,0,0,0,0
4,2020-01-07,AF,Afghanistan,EMRO,0,0,0,0
...,...,...,...,...,...,...,...,...
214006,2022-06-19,ZW,Zimbabwe,AFRO,157,254801,5,5534
214007,2022-06-20,ZW,Zimbabwe,AFRO,30,254831,0,5534
214008,2022-06-21,ZW,Zimbabwe,AFRO,88,254919,2,5536
214009,2022-06-22,ZW,Zimbabwe,AFRO,0,254919,0,5536


In [13]:
now_df_daily_cd['Country'].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Andorra',
       'Angola', 'Anguilla', 'Antigua and Barbuda', 'Argentina',
       'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan',
       'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus',
       'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan',
       'Bolivia (Plurinational State of)', 'Bonaire',
       'Bosnia and Herzegovina', 'Botswana', 'Brazil',
       'British Virgin Islands', 'Brunei Darussalam', 'Bulgaria',
       'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cambodia', 'Cameroon',
       'Canada', 'Cayman Islands', 'Central African Republic', 'Chad',
       'Chile', 'China', 'Colombia', 'Comoros', 'Congo', 'Cook Islands',
       'Costa Rica', 'Côte d’Ivoire', 'Croatia', 'Cuba', 'Curaçao',
       'Cyprus', 'Czechia', "Democratic People's Republic of Korea",
       'Democratic Republic of the Congo', 'Denmark', 'Djibouti',
       'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt',
       'El Salvado

In [14]:
now_df_daily_cd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214011 entries, 0 to 214010
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Date_reported      214011 non-null  object
 1   Country_code       213108 non-null  object
 2   Country            214011 non-null  object
 3   WHO_region         214011 non-null  object
 4   New_cases          214011 non-null  int64 
 5   Cumulative_cases   214011 non-null  int64 
 6   New_deaths         214011 non-null  int64 
 7   Cumulative_deaths  214011 non-null  int64 
dtypes: int64(4), object(4)
memory usage: 13.1+ MB


We have to change the Date_reported dtypes to date.

## Latest Case and Death

In [15]:
now_df_latest_cd

,Name,WHO Region,Cases - cumulative total,Cases - cumulative total per 100000 population,Cases - newly reported in last 7 days,Cases - newly reported in last 7 days per 100000 population,Cases - newly reported in last 24 hours,Deaths - cumulative total,Deaths - cumulative total per 100000 population,Deaths - newly reported in last 7 days,Deaths - newly reported in last 7 days per 100000 population,Deaths - newly reported in last 24 hours
0,Global,NaN,199466211,2559.050333,4158340,53.349393,548167,4244541,54.455309,64036,0.821549,8430
1,United States of America,Americas,35010407,10577.080000,618994,187.010000,78722,609022,183.990000,2759,0.830000,387
2,India,South-East Asia,31769132,2302.100000,284527,20.620000,42625,425757,30.850000,3735,0.270000,562
3,Brazil,Americas,19953501,9387.260000,245839,115.660000,15143,557223,262.150000,6721,3.160000,389
4,Russian Federation,Europe,6356784,4355.920000,161552,110.700000,22589,161715,110.810000,5537,3.790000,790
...,...,...,...,...,...,...,...,...,...,...,...,...
233,Saint Helena,Africa,0,0.000000,0,0.000000,0,0,0.000000,0,0.000000,0
234,Tokelau,Western Pacific,0,0.000000,0,0.000000,0,0,0.000000,0,0.000000,0
235,Tonga,Western Pacific,0,0.000000,0,0.000000,0,0,0.000000,0,0.000000,0
236,Turkmenistan,Europe,0,0.000000,0,0.000000,0,0,0.000000,0,0.000000,0


In [16]:
now_df_latest_cd = now_df_latest_cd.rename(columns = {'Name': 'Country'})

In [17]:
now_df_latest_cd['Country'].unique()

array(['Global', 'United States of America', 'India', 'Brazil',
       'Russian Federation', 'France', 'The United Kingdom', 'Turkey',
       'Argentina', 'Colombia', 'Spain', 'Italy',
       'Iran (Islamic Republic of)', 'Germany', 'Indonesia', 'Poland',
       'Mexico', 'South Africa', 'Ukraine', 'Peru', 'Netherlands',
       'Czechia', 'Iraq', 'Chile', 'Philippines', 'Canada', 'Bangladesh',
       'Malaysia', 'Belgium', 'Sweden', 'Romania', 'Pakistan', 'Portugal',
       'Japan', 'Israel', 'Hungary', 'Jordan', 'Serbia', 'Switzerland',
       'Nepal', 'United Arab Emirates', 'Thailand', 'Austria', 'Morocco',
       'Kazakhstan', 'Tunisia', 'Lebanon', 'Saudi Arabia', 'Greece',
       'Ecuador', 'Bolivia (Plurinational State of)', 'Paraguay',
       'Belarus', 'Panama', 'Georgia', 'Bulgaria', 'Cuba', 'Costa Rica',
       'Kuwait', 'Slovakia', 'Uruguay', 'Guatemala', 'Croatia',
       'occupied Palestinian territory, including east Jerusalem',
       'Azerbaijan', 'Dominican Republic', 

In [18]:
now_df_latest_cd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 238 entries, 0 to 237
Data columns (total 12 columns):
 #   Column                                                        Non-Null Count  Dtype  
---  ------                                                        --------------  -----  
 0   Country                                                       238 non-null    object 
 1   WHO Region                                                    237 non-null    object 
 2   Cases - cumulative total                                      238 non-null    int64  
 3   Cases - cumulative total per 100000 population                237 non-null    float64
 4   Cases - newly reported in last 7 days                         238 non-null    int64  
 5   Cases - newly reported in last 7 days per 100000 population   237 non-null    float64
 6   Cases - newly reported in last 24 hours                       238 non-null    int64  
 7   Deaths - cumulative total                                     238 non-n

We will drop the 'Global' value as it is not neccessary.

In [19]:
now_df_latest_cd = now_df_latest_cd[now_df_latest_cd['Country'] != 'Global']

## Vaccination

In [20]:
now_df_vaccination

,COUNTRY,ISO3,WHO_REGION,DATA_SOURCE,DATE_UPDATED,TOTAL_VACCINATIONS,PERSONS_VACCINATED_1PLUS_DOSE,TOTAL_VACCINATIONS_PER100,PERSONS_VACCINATED_1PLUS_DOSE_PER100,PERSONS_FULLY_VACCINATED,PERSONS_FULLY_VACCINATED_PER100,VACCINES_USED,FIRST_VACCINE_DATE,NUMBER_VACCINES_TYPES_USED,PERSONS_BOOSTER_ADD_DOSE,PERSONS_BOOSTER_ADD_DOSE_PER100
0,Afghanistan,AFG,EMRO,REPORTING,2022-06-21,6355931,5629522.0,16.327,14.461,4979930.0,12.793,"AstraZeneca - Vaxzevria,Beijing CNBG - BBIBP-C...",2021-02-22,11.0,NaN,NaN
1,Albania,ALB,EURO,REPORTING,2022-06-05,2883079,1322864.0,100.200,46.482,1244383.0,43.725,"AstraZeneca - Vaxzevria,Gamaleya - Gam-Covid-V...",2021-01-13,5.0,313660.0,11.021
2,Algeria,DZA,AFRO,REPORTING,2022-06-05,15205854,7840131.0,34.676,17.879,6481186.0,14.780,"Beijing CNBG - BBIBP-CorV,Gamaleya - Gam-Covid...",2021-01-30,4.0,514063.0,1.172
3,American Samoa,ASM,WPRO,REPORTING,2022-06-12,108220,44456.0,196.061,80.541,40945.0,74.180,"Janssen - Ad26.COV 2-S,Moderna - Spikevax,Pfiz...",2020-12-21,3.0,22893.0,41.475
4,Andorra,AND,EURO,REPORTING,2022-05-29,153072,57880.0,198.100,75.981,53450.0,70.166,"AstraZeneca - Vaxzevria,Moderna - Spikevax,Pfi...",2021-01-20,3.0,41742.0,54.796
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,Viet Nam,VNM,WPRO,REPORTING,2022-06-12,223070073,84922250.0,229.169,87.244,77994241.0,80.127,"AstraZeneca - Vaxzevria,Beijing CNBG - BBIBP-C...",2021-03-08,6.0,57883770.0,59.466
224,Wallis and Futuna,WLF,WPRO,REPORTING,2022-06-12,16426,6592.0,146.061,58.616,6633.0,58.981,Moderna - Spikevax,2021-03-19,1.0,3201.0,28.463
225,Yemen,YEM,EMRO,REPORTING,2022-06-19,864544,697956.0,2.899,2.340,446120.0,1.496,"AstraZeneca - Vaxzevria,Beijing CNBG - BBIBP-C...",2021-04-20,11.0,80.0,0.000
226,Zambia,ZMB,AFRO,REPORTING,2022-06-05,3750417,4656681.0,20.400,25.330,2935333.0,15.967,"Beijing CNBG - BBIBP-CorV,Janssen - Ad26.COV 2...",2021-04-14,3.0,95513.0,0.520


In [21]:
now_df_vaccination.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228 entries, 0 to 227
Data columns (total 16 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   COUNTRY                               228 non-null    object 
 1   ISO3                                  228 non-null    object 
 2   WHO_REGION                            228 non-null    object 
 3   DATA_SOURCE                           228 non-null    object 
 4   DATE_UPDATED                          228 non-null    object 
 5   TOTAL_VACCINATIONS                    228 non-null    int64  
 6   PERSONS_VACCINATED_1PLUS_DOSE         227 non-null    float64
 7   TOTAL_VACCINATIONS_PER100             228 non-null    float64
 8   PERSONS_VACCINATED_1PLUS_DOSE_PER100  227 non-null    float64
 9   PERSONS_FULLY_VACCINATED              227 non-null    float64
 10  PERSONS_FULLY_VACCINATED_PER100       227 non-null    float64
 11  VACCINES_USED      

We have to change the DATE_UPDATED and FIRST_VACCINE_DATE dtypes to date.

## Vaccination

In [22]:
now_df_vaccination_md

,ISO3,VACCINE_NAME,PRODUCT_NAME,COMPANY_NAME,AUTHORIZATION_DATE,START_DATE,END_DATE,COMMENT,DATA_SOURCE
0,FLK,AstraZeneca - AZD1222,AZD1222,AstraZeneca,NaN,NaN,NaN,NaN,OWID
1,SHN,AstraZeneca - AZD1222,AZD1222,AstraZeneca,NaN,NaN,NaN,NaN,OWID
2,GRL,Moderna - mRNA-1273,mRNA-1273,Moderna,NaN,NaN,NaN,NaN,OWID
3,FRO,Moderna - mRNA-1273,mRNA-1273,Moderna,NaN,NaN,NaN,NaN,OWID
4,FRO,Pfizer BioNTech - Comirnaty,Comirnaty,Pfizer BioNTech,NaN,NaN,NaN,NaN,OWID
...,...,...,...,...,...,...,...,...,...
1041,SWE,Unknown Vaccine,Unknown Vaccine,NaN,NaN,2020-12-23,NaN,NaN,REPORTING
1042,TKM,Unknown Vaccine,Unknown Vaccine,NaN,NaN,2022-03-02,NaN,NaN,REPORTING
1043,MCO,Novavax - Covavax,Covavax,NaN,NaN,2022-03-30,NaN,NaN,REPORTING
1044,THA,SII - Covovax,Covovax,NaN,2022-04-27,NaN,NaN,NaN,REPORTING


In [23]:
now_df_vaccination_md.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1046 entries, 0 to 1045
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ISO3                1046 non-null   object 
 1   VACCINE_NAME        1046 non-null   object 
 2   PRODUCT_NAME        1045 non-null   object 
 3   COMPANY_NAME        1027 non-null   object 
 4   AUTHORIZATION_DATE  488 non-null    object 
 5   START_DATE          762 non-null    object 
 6   END_DATE            0 non-null      float64
 7   COMMENT             0 non-null      float64
 8   DATA_SOURCE         1046 non-null   object 
dtypes: float64(2), object(7)
memory usage: 73.7+ KB


:We have to change the START_DATE and END_DATE dtypes to date.

# Data Cleaning

In [24]:
print("================================")
print("Cleaning the Data .....")
print()

Cleaning the Data .....



## Set up columns name

We want our format name to be readable in SQL format as we will import the dataframe to our local DB.

In [26]:
# Lower the words
# Replace any symbols except ' with _
# Delete any space
def remove_symbols(text):
    # Deleting extra spaces and lowering the char, as well as removing ' symbol
    replacement = {"`": "",
                   "'": "",
                   '"': ""}

    temp_text = re.sub(' {2,}', ' ', text.lower().strip().replace("`", "").replace('"',""))

    return re.sub("[^0-9a-zA-Z]+", "_", temp_text)

## Check Values

Though it is unlikely, we will make sure that there will not be any negative value(s) in the dataframe as well as numbers in the country names.

In [25]:
# Check negative values
for df in [now_df_daily_cd, now_df_latest_cd, now_df_vaccination, now_df_vaccination_md]:
    for col in df.columns:
        if (df[col].dtypes == 'int64') or (df[col].dtypes == 'float64'):
            df[col] = df[col].apply(abs)

In [90]:
# Get only the alphabetical
def get_alphabet(text):
    arr = re.findall(r'[A-Za-z]+', text)
    return ''.join(arr)

for df in [now_df_daily_cd, now_df_latest_cd, now_df_vaccination, now_df_vaccination_md]:
    for col in ['country', 'country_code', 'iso3']:
        try:
            df[col] = df[col].apply(get_alphabet)
            
        except Exception as e:
            pass

At this point, we can be sure that there will be no numbers or symbols in our country, country_code, and iso3 columns.

## Date parsing

Make sure that all the data format are homogenous.

In [27]:
# Parse all the date format in case of different format from multiple sources
# NOTE: This function is used when data from multiple sources are used. For this case,
# where data is coming from single source(WHO), it most likely will not be of use.
def date_parsing(text):    
    fmts = ('%Y', '%b %d, %Y','%b %d,%Y', '%B %d, %Y', '%B %d %Y',
            '%d %b, %Y', '%d %b,%Y', '%d %b %Y', '%d %B %Y', '%b %d, %Y',
            '%b %d %Y', '%b %d,%Y', '%Y %b %d', '%Y %B %d', '%d/%m/%Y', '%d/%m/%y',
            '%b %Y', '%B%Y', '%b %d,%Y', '%Y/%m/%d', '%y/%m/%d', '%Y %b',
            '%Y, %b %d', '%Y, %B %d', '%Y, %d %b', '%Y, %d %B', '%Y %d %b', '%Y %d %B')
    
    for fmt in fmts:
        try:
            temp_text = re.sub(' {2,}', ' ', text.strip().replace('-', '/'))
            parsed = dt.datetime.strptime(temp_text, fmt)
            return parsed

        except Exception as e:
            return text

In [28]:
# Daily Case and Death
now_df_daily_cd.columns = now_df_daily_cd.columns.to_series().apply(remove_symbols)
for col in now_df_daily_cd.columns:
    if now_df_daily_cd[col].dtype == 'object':
        now_df_daily_cd[col] = now_df_daily_cd[col].apply(date_parsing)

# Latest Case and Death
now_df_latest_cd.columns = now_df_latest_cd.columns.to_series().apply(remove_symbols)
for col in now_df_latest_cd.columns:
    if now_df_latest_cd[col].dtype == 'object':
        now_df_latest_cd[col] = now_df_latest_cd[col].apply(date_parsing)

# Vaccination
now_df_vaccination.columns = now_df_vaccination.columns.to_series().apply(remove_symbols)
for col in now_df_vaccination.columns:
    if now_df_vaccination[col].dtype == 'object':
        now_df_vaccination[col] = now_df_vaccination[col].apply(date_parsing)

# Vaccination Metadata
now_df_vaccination_md.columns = now_df_vaccination_md.columns.to_series().apply(remove_symbols)
for col in now_df_vaccination_md.columns:
    if now_df_vaccination_md[col].dtype == 'object':
        now_df_vaccination_md[col] = now_df_vaccination_md[col].apply(date_parsing)

## Datatypes

In [29]:
# Daily Case and Death
for col in now_df_daily_cd.columns:
    print(f"\n======================= {col} =======================")
    display(now_df_daily_cd[col].value_counts())


======================= date_reported =======================


2020-01-03    237
2021-07-21    237
2021-08-20    237
2021-08-21    237
2021-08-22    237
             ... 
2020-11-03    237
2020-11-04    237
2020-11-05    237
2020-11-06    237
2022-06-23    237
Name: date_reported, Length: 903, dtype: int64


======================= country_code =======================


AF    903
PY    903
NG    903
NU    903
MK    903
     ... 
GL    903
GD    903
GP    903
GU    903
ZW    903
Name: country_code, Length: 236, dtype: int64


======================= country =======================


...

0         26593
1          1523
3          1057
26          795
5           780
          ...  
202722        1
4333          1
202877        1
4234          1
59932         1
Name: cumulative_cases, Length: 99304, dtype: int64


======================= new_deaths =======================


0       124580
1        13129
2         7937
3         5773
4         4467
         ...  
1587         1
1647         1
1576         1
1422         1
1502         1
Name: new_deaths, Length: 1681, dtype: int64


======================= cumulative_deaths =======================


0        45418
1         4822
2         2684
3         2169
7         1456
         ...  
59117        1
59028        1
58945        1
58883        1
5498         1
Name: cumulative_deaths, Length: 33272, dtype: int64

In [30]:
# Latest Case and Death
for col in now_df_latest_cd.columns:
    print(f"\n======================= {col} =======================")
    display(now_df_latest_cd[col].value_counts())


======================= country =======================


United States of America    1
Guam                        1
Belize                      1
Curaçao                     1
Lesotho                     1
                           ..
Republic of Korea           1
Zambia                      1
Nigeria                     1
Algeria                     1
Tuvalu                      1
Name: country, Length: 237, dtype: int64


======================= who_region =======================


Europe                   62
Americas                 56
Africa                   50
Western Pacific          35
Eastern Mediterranean    22
South-East Asia          11
Other                     1
Name: who_region, dtype: int64


======================= cases_cumulative_total =======================


...

0      69
1      21
2       9
4       8
3       6
       ..
92      1
68      1
90      1
442     1
72      1
Name: deaths_newly_reported_in_last_7_days, Length: 102, dtype: int64


======================= deaths_newly_reported_in_last_7_days_per_100000_population =======================


0.00    70
0.02    10
0.01     7
0.04     5
0.03     4
        ..
4.94     1
1.72     1
0.74     1
5.40     1
1.55     1
Name: deaths_newly_reported_in_last_7_days_per_100000_population, Length: 113, dtype: int64


======================= deaths_newly_reported_in_last_24_hours =======================


0       121
1        15
4        10
2         8
5         6
3         6
9         5
7         4
11        3
15        3
6         3
80        2
35        2
16        2
25        2
10        2
41        2
8         2
12        2
64        1
188       1
14        1
387       1
20        1
44        1
322       1
68        1
30        1
190       1
32        1
21        1
29        1
38        1
19        1
562       1
67        1
389       1
790       1
62        1
138       1
126       1
273       1
218       1
18        1
27        1
378       1
1747      1
245       1
555       1
43        1
63        1
24        1
48        1
241       1
195       1
13        1
Name: deaths_newly_reported_in_last_24_hours, dtype: int64

In [31]:
# Vaccination
for col in now_df_vaccination.columns:
    print(f"\n======================= {col} =======================")
    display(now_df_vaccination[col].value_counts())


======================= country =======================


Afghanistan    1
Netherlands    1
New Zealand    1
Nicaragua      1
Niger          1
              ..
Gibraltar      1
Greece         1
Greenland      1
Grenada        1
Zimbabwe       1
Name: country, Length: 228, dtype: int64


======================= iso3 =======================


AFG    1
NLD    1
NZL    1
NIC    1
NER    1
      ..
GIB    1
GRC    1
GRL    1
GRD    1
ZWE    1
Name: iso3, Length: 228, dtype: int64


======================= who_region =======================


...

5.0     39
3.0     36
2.0     32
4.0     30
6.0     22
1.0     21
11.0    11
8.0     11
10.0    11
7.0     10
12.0     1
9.0      1
Name: number_vaccines_types_used, dtype: int64


======================= persons_booster_add_dose =======================


0.0          2
313660.0     1
1615301.0    1
942369.0     1
1094.0       1
            ..
84781.0      1
61789.0      1
3034134.0    1
81.0         1
829558.0     1
Name: persons_booster_add_dose, Length: 196, dtype: int64


======================= persons_booster_add_dose_per100 =======================


0.000     3
0.087     2
0.004     2
11.021    1
0.457     1
         ..
5.994     1
21.189    1
36.610    1
16.936    1
5.581     1
Name: persons_booster_add_dose_per100, Length: 193, dtype: int64

In [32]:
# Vaccination Metadata
for col in now_df_vaccination_md.columns:
    print(f"\n======================= {col} =======================")
    display(now_df_vaccination_md[col].value_counts())


======================= iso3 =======================


IRN    12
PHL    11
SOM    11
ARE    11
YEM    11
       ..
BDI     1
TCD     1
RUS     1
GRL     1
LBR     1
Name: iso3, Length: 225, dtype: int64


======================= vaccine_name =======================


Pfizer BioNTech - Comirnaty    171
AstraZeneca - Vaxzevria        138
Janssen - Ad26.COV 2-S         122
Moderna - Spikevax             119
Beijing CNBG - BBIBP-CorV       99
SII - Covishield                91
Gamaleya - Gam-Covid-Vac        68
Sinovac - CoronaVac             64
Bharat - Covaxin                33
Gamaleya - Sputnik-Light        31
CanSino - Convidecia            28
Novavax-NUVAXOVID               25
Unknown Vaccine                 11
AstraZeneca - AZD1222            7
Moderna - mRNA-1273              5
Finlay - Soberana-02             4
CIGB - CIGB-66                   4
Julphar - Hayat-Vax              3
Wuhan CNBG - Inactivated         3
RIBSP - QazVac                   3
Anhui ZL - Zifivax               3
SII - Covovax                    2
SRCVB - EpiVacCorona             2
Finlay - Soberana Plus           2
Novavax - Covavax                1
Shifa - COVIran Barakat          1
Biological E - Corbevax          1
Turkovac                         1
IMB - Covidful      


======================= product_name =======================


...

Series([], Name: end_date, dtype: int64)


======================= comment =======================


Series([], Name: comment, dtype: int64)


======================= data_source =======================


REPORTING    1023
OWID           23
Name: data_source, dtype: int64

In [33]:
# Daily Case and Death Data
now_df_daily_cd['date_reported'] = pd.to_datetime(now_df_daily_cd['date_reported'])

# Latest Case and Death Data
# -

# Vaccination Data
now_df_vaccination['date_updated'] = pd.to_datetime(now_df_vaccination['date_updated'])
now_df_vaccination['first_vaccine_date'] = pd.to_datetime(now_df_vaccination['first_vaccine_date'])

# Vaccination Metadata
now_df_vaccination_md['start_date'] = pd.to_datetime(now_df_vaccination_md['start_date'])
now_df_vaccination_md['end_date'] = pd.to_datetime(now_df_vaccination_md['end_date'])

In [34]:
print("Successfully cleaned the Data.")
print("================================")
print()

Successfully cleaned the Data.



# Handle Missing Value(s)

Notice that as null values are most probably indicating that there is no report for the given column/criteria at the specific period, then the most logical thing to do leave these null values as it is.

In [35]:
# -

# Handle Duplicate Value(s)

## WHO Data

In [36]:
# Daily Case and Death Data
now_df_daily_cd = now_df_daily_cd.drop_duplicates()

# Latest Case and Death Data
now_df_latest_cd = now_df_latest_cd.drop_duplicates()

# Vaccination Data
now_df_vaccination = now_df_vaccination.drop_duplicates()

# Vaccination Metadata
now_df_vaccination_md = now_df_vaccination_md.drop_duplicates()

# Data Manipulation

To make our format consistent, we will do the followings::
1.  Round up some float numbers to only 3 decimal numbers. We do this because want our data to be as readable as we can. Thus, we need to do some manipulation to the data.
2.  Add countrycode feature if it does not exists and convert ISO3 into country code feature, then re-order the columns.
3.  Match each country's name in the database with the names written in the dataframe to avoid any ambiguation.
4.  Add feature(s) to make sure that the data is not ambiguous.

In [37]:
print("================================")
print("Manipulating the Data .....")
print()

Manipulating the Data .....



## Round Decimals

In [38]:
print("Rounding Decimals .....")
print()

Rounding Decimals .....



In [39]:
# Daily Case and Death
for col in now_df_daily_cd.columns:
    if now_df_daily_cd[col].dtypes == 'float64':
        now_df_daily_cd[col] = now_df_daily_cd[col].apply(lambda x: round(x, 3))
    
# Latest Case and Death Data
for col in now_df_latest_cd.columns:
    if now_df_latest_cd[col].dtypes == 'float64':
        now_df_latest_cd[col] = now_df_latest_cd[col].apply(lambda x: round(x, 3))
    
# Vaccination Data
for col in now_df_vaccination.columns:
    if now_df_vaccination[col].dtypes == 'float64':
        now_df_vaccination[col] = now_df_vaccination[col].apply(lambda x: round(x, 3))
    
# Vaccination Metadata
for col in now_df_vaccination_md.columns:
    if now_df_vaccination_md[col].dtypes == 'float64':
        now_df_vaccination_md[col] = now_df_vaccination_md[col].apply(lambda x: round(x, 3))

## Match Country Name(s) - 1st Layer

In [40]:
print("Matching Country Name(s) - 1st Layer .....")
print()

Matching Country Name(s) - 1st Layer .....



In [41]:
# Get country name from ISO3
country_iso3_mapping = {country.alpha_3: country.name for country in pycountry.countries}

# Get country code from country name
country_code_mapping = {country.name: country.alpha_2 for country in pycountry.countries}

### Daily Case and Death

In [42]:
now_df_daily_cd.columns

Index(['date_reported', 'country_code', 'country', 'who_region', 'new_cases',
       'cumulative_cases', 'new_deaths', 'cumulative_deaths'],
      dtype='object')

In [43]:
now_df_daily_cd[now_df_daily_cd[['country_code', 'country', 'who_region']]
                .isna().any(axis = 1)].country

129129    Namibia
129130    Namibia
129131    Namibia
129132    Namibia
129133    Namibia
           ...   
130027    Namibia
130028    Namibia
130029    Namibia
130030    Namibia
130031    Namibia
Name: country, Length: 903, dtype: object

In [44]:
# Store the countries with NaN value
nan_now_df_daily_cd = now_df_daily_cd[now_df_daily_cd[['country_code', 'country', 'who_region']]
                                      .isna().any(axis = 1)].country

### Latest Case and Death

In [45]:
now_df_latest_cd.columns

Index(['country', 'who_region', 'cases_cumulative_total',
       'cases_cumulative_total_per_100000_population',
       'cases_newly_reported_in_last_7_days',
       'cases_newly_reported_in_last_7_days_per_100000_population',
       'cases_newly_reported_in_last_24_hours', 'deaths_cumulative_total',
       'deaths_cumulative_total_per_100000_population',
       'deaths_newly_reported_in_last_7_days',
       'deaths_newly_reported_in_last_7_days_per_100000_population',
       'deaths_newly_reported_in_last_24_hours'],
      dtype='object')

In [46]:
now_df_latest_cd['country_code'] = now_df_latest_cd['country'].apply(lambda x: country_code_mapping.get(x))
now_df_latest_cd = now_df_latest_cd[['country_code', 'country', 'who_region', 
                                     'cases_cumulative_total', 'cases_cumulative_total_per_100000_population',
                                     'cases_newly_reported_in_last_7_days',
                                     'cases_newly_reported_in_last_7_days_per_100000_population',
                                     'cases_newly_reported_in_last_24_hours', 'deaths_cumulative_total',
                                     'deaths_cumulative_total_per_100000_population',
                                     'deaths_newly_reported_in_last_7_days',
                                     'deaths_newly_reported_in_last_7_days_per_100000_population',
                                     'deaths_newly_reported_in_last_24_hours']]

now_df_latest_cd[now_df_latest_cd[['country_code', 'country', 'who_region']]
                 .isna().any(axis = 1)]

,country_code,country,who_region,cases_cumulative_total,cases_cumulative_total_per_100000_population,cases_newly_reported_in_last_7_days,cases_newly_reported_in_last_7_days_per_100000_population,cases_newly_reported_in_last_24_hours,deaths_cumulative_total,deaths_cumulative_total_per_100000_population,deaths_newly_reported_in_last_7_days,deaths_newly_reported_in_last_7_days_per_100000_population,deaths_newly_reported_in_last_24_hours
1,None,United States of America,Americas,35010407,10577.08,618994,187.01,78722,609022,183.99,2759,0.83,387
6,None,The United Kingdom,Europe,5923824,8726.13,178294,262.64,21466,129881,191.32,578,0.85,138
12,None,Iran (Islamic Republic of),Eastern Mediterranean,3979727,4738.17,221530,263.75,39019,91785,109.28,2306,2.75,378
50,None,Bolivia (Plurinational State of),Americas,474538,4065.25,5356,45.88,639,17859,152.99,187,1.60,20
63,None,"occupied Palestinian territory, including east...",Eastern Mediterranean,346105,6784.49,936,18.35,181,3877,76.00,8,0.16,0
69,None,Venezuela (Bolivarian Republic of),Americas,307570,1081.62,6651,23.39,897,3623,12.74,114,0.40,16
77,None,Republic of Moldova,Europe,259828,6441.01,976,24.19,161,6261,155.21,14,0.35,4
84,None,Republic of Korea,Western Pacific,203926,397.76,10501,20.48,1725,2106,4.11,23,0.04,2
105,None,Kosovo[1],Europe,108580,6046.78,550,30.63,110,2257,125.69,2,0.11,1
120,None,Democratic Republic of the Congo,Africa,50529,56.42,1761,1.97,336,1045,1.17,22,0.02,7


In [47]:
# Store the countries with NaN value
nan_now_df_latest_cd = now_df_latest_cd[now_df_latest_cd[['country_code', 'country', 'who_region']]
                                        .isna().any(axis = 1)].country

### Vaccination

In [48]:
now_df_vaccination.columns

Index(['country', 'iso3', 'who_region', 'data_source', 'date_updated',
       'total_vaccinations', 'persons_vaccinated_1plus_dose',
       'total_vaccinations_per100', 'persons_vaccinated_1plus_dose_per100',
       'persons_fully_vaccinated', 'persons_fully_vaccinated_per100',
       'vaccines_used', 'first_vaccine_date', 'number_vaccines_types_used',
       'persons_booster_add_dose', 'persons_booster_add_dose_per100'],
      dtype='object')

In [49]:
now_df_vaccination['country'] = now_df_vaccination['iso3'].apply(lambda x: country_iso3_mapping.get(x))
now_df_vaccination['country_code'] = now_df_vaccination['country'].apply(lambda x: country_code_mapping.get(x))
now_df_vaccination = now_df_vaccination[['country_code', 'country', 'who_region', 'data_source', 
                                         'date_updated', 'total_vaccinations', 'persons_vaccinated_1plus_dose',
                                         'total_vaccinations_per100', 'persons_vaccinated_1plus_dose_per100',
                                         'persons_fully_vaccinated', 'persons_fully_vaccinated_per100',
                                         'vaccines_used', 'first_vaccine_date', 'number_vaccines_types_used',
                                         'persons_booster_add_dose', 'persons_booster_add_dose_per100',]]

now_df_vaccination[now_df_vaccination[['country_code', 'country', 'who_region', 
                                       'data_source', 'date_updated']].isna().any(axis = 1)]

,country_code,country,who_region,data_source,date_updated,total_vaccinations,persons_vaccinated_1plus_dose,total_vaccinations_per100,persons_vaccinated_1plus_dose_per100,persons_fully_vaccinated,persons_fully_vaccinated_per100,vaccines_used,first_vaccine_date,number_vaccines_types_used,persons_booster_add_dose,persons_booster_add_dose_per100
25,None,None,AMRO,REPORTING,2022-06-17,41141,17414.0,196.706,83.261,15835.0,75.711,NaN,NaT,NaN,7892.0,37.734
109,None,None,EURO,REPORTING,2022-05-29,1828696,903026.0,101.800,50.289,822798.0,45.821,"AstraZeneca - Vaxzevria,Pfizer BioNTech - Comi...",2021-03-31,2.0,102872.0,5.729
172,None,None,AMRO,REPORTING,2022-06-17,4419,1680.0,228.608,86.912,1639.0,84.790,NaN,NaT,NaN,1100.0,56.906
186,None,None,AMRO,REPORTING,2022-06-17,4257,1791.0,135.616,57.056,1686.0,53.711,NaN,NaT,NaN,780.0,24.849


In [50]:
# Store the countries with NaN value
nan_now_df_vaccination = now_df_vaccination[now_df_vaccination[['country_code', 'country', 
                                                                'who_region', 'data_source', 'date_updated']]
                                            .isna().any(axis = 1)].country

### Vaccination Metadata

In [51]:
now_df_vaccination_md.columns

Index(['iso3', 'vaccine_name', 'product_name', 'company_name',
       'authorization_date', 'start_date', 'end_date', 'comment',
       'data_source'],
      dtype='object')

In [52]:
now_df_vaccination_md['country'] = now_df_vaccination_md['iso3'].apply(lambda x: country_iso3_mapping.get(x))
now_df_vaccination_md['country_code'] = now_df_vaccination_md['country'].apply(lambda x: country_code_mapping.get(x))
now_df_vaccination_md = now_df_vaccination_md[['country_code', 'country', 'vaccine_name', 'product_name', 'company_name',
                                               'authorization_date', 'start_date', 'end_date', 'comment', 'data_source']]

now_df_vaccination_md[now_df_vaccination_md[['country_code', 'vaccine_name']].isna().any(axis = 1)]

,country_code,country,vaccine_name,product_name,company_name,authorization_date,start_date,end_date,comment,data_source
289,None,None,AstraZeneca - Vaxzevria,Vaxzevria,AstraZeneca,NaN,2021-03-31,NaT,NaN,REPORTING
460,None,None,Pfizer BioNTech - Comirnaty,Comirnaty,Pfizer BioNTech,NaN,2021-05-05,NaT,NaN,REPORTING


In [53]:
# Store the countries with NaN value
nan_now_df_vaccination_md = now_df_vaccination_md[now_df_vaccination_md[['country_code', 'vaccine_name']]
                                               .isna().any(axis = 1)].country

## Match Country Name(s) - 2nd Layer

In [54]:
print("Matching Country Name(s) - 2nd Layer .....")
print()

Matching Country Name(s) - 2nd Layer .....



In [55]:
# List of all countries in the python modules
countries = [country.name for country in pycountry.countries]
countries

['Aruba',
 'Afghanistan',
 'Angola',
 'Anguilla',
 'Åland Islands',
 'Albania',
 'Andorra',
 'United Arab Emirates',
 'Argentina',
 'Armenia',
 'American Samoa',
 'Antarctica',
 'French Southern Territories',
 'Antigua and Barbuda',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Burundi',
 'Belgium',
 'Benin',
 'Bonaire, Sint Eustatius and Saba',
 'Burkina Faso',
 'Bangladesh',
 'Bulgaria',
 'Bahrain',
 'Bahamas',
 'Bosnia and Herzegovina',
 'Saint Barthélemy',
 'Belarus',
 'Belize',
 'Bermuda',
 'Bolivia, Plurinational State of',
 'Brazil',
 'Barbados',
 'Brunei Darussalam',
 'Bhutan',
 'Bouvet Island',
 'Botswana',
 'Central African Republic',
 'Canada',
 'Cocos (Keeling) Islands',
 'Switzerland',
 'Chile',
 'China',
 "Côte d'Ivoire",
 'Cameroon',
 'Congo, The Democratic Republic of the',
 'Congo',
 'Cook Islands',
 'Colombia',
 'Comoros',
 'Cabo Verde',
 'Costa Rica',
 'Cuba',
 'Curaçao',
 'Christmas Island',
 'Cayman Islands',
 'Cyprus',
 'Czechia',
 'Germany',
 'Djibouti',
 'Dominica'

In [56]:
# Swap the word position if contains comma
def swap_text(text):
    if ',' in text:
        return (text.split(',')[1] + ' ' + text.split(',')[0]).strip()
    
    else:
        return text

# New dictionary for matching values
country_iso3_mapping_new = {}
country_code_mapping_new = {}
    
# Get country name from ISO3
for key, value in country_iso3_mapping.items():
    country_iso3_mapping_new[key] = swap_text(value)

# Get country code from country name
for key in country_code_mapping.keys():
    country_code_mapping_new[swap_text(key)] = country_code_mapping[key]
    
countries = list(country_code_mapping_new.keys())

### Search for word with highest similarity

In [57]:
now_df_daily_cd[now_df_daily_cd['country_code'].isna() == 1]

,date_reported,country_code,country,who_region,new_cases,cumulative_cases,new_deaths,cumulative_deaths
129129,2020-01-03,NaN,Namibia,AFRO,0,0,0,0
129130,2020-01-04,NaN,Namibia,AFRO,0,0,0,0
129131,2020-01-05,NaN,Namibia,AFRO,0,0,0,0
129132,2020-01-06,NaN,Namibia,AFRO,0,0,0,0
129133,2020-01-07,NaN,Namibia,AFRO,0,0,0,0
...,...,...,...,...,...,...,...,...
130027,2022-06-19,NaN,Namibia,AFRO,0,165783,0,4054
130028,2022-06-20,NaN,Namibia,AFRO,0,165783,0,4054
130029,2022-06-21,NaN,Namibia,AFRO,0,165783,0,4054
130030,2022-06-22,NaN,Namibia,AFRO,0,165783,0,4054


In [58]:
# Levenshtein distance to calculate the distance between 2 words (checking 2 words similarity)
def levenshteinDistanceDP(token1, token2):
    distances = np.zeros((len(token1) + 1, len(token2) + 1))

    for t1 in range(len(token1) + 1):
        distances[t1][0] = t1

    for t2 in range(len(token2) + 1):
        distances[0][t2] = t2
        
    a = 0
    b = 0
    c = 0
    
    for t1 in range(1, len(token1) + 1):
        for t2 in range(1, len(token2) + 1):
            if (token1[t1-1] == token2[t2-1]):
                distances[t1][t2] = distances[t1 - 1][t2 - 1]
            else:
                a = distances[t1][t2 - 1]
                b = distances[t1 - 1][t2]
                c = distances[t1 - 1][t2 - 1]
                
                if (a <= b and a <= c):
                    distances[t1][t2] = a + 1
                elif (b <= a and b <= c):
                    distances[t1][t2] = b + 1
                else:
                    distances[t1][t2] = c + 1

    return distances[len(token1)][len(token2)]

In [59]:
# Find the word with the highest similarity
def most_similar(text):
    idx = 0
    score = [levenshteinDistanceDP(text, countries[i]) for i in range(len(countries))]
    
    # Word with the most similarity has the least score
    minv = min(score)
    
    # As the index of country in both list are the same, this is acceptable
    return countries[score.index(minv)] 

### Check NaN entries (country)

In [ ]:
for item in [nan_now_df_daily_cd, nan_now_df_latest_cd, nan_now_df_vaccination, nan_now_df_vaccination_md]:
    for idx, value in enumerate(item):
        try:
            print(f"{value} => \t{most_similar(value)} => \t{country_code_mapping_new[most_similar(value)]}")
            
        except Exception as e:
            print(e)

We will exclude the values which contains 'Saint', 'Sint', 'None', 'Other', and 'Kosovo'.
We also will change the Palestinian name into State of Palestinian and Jerusalem.

In [73]:
for item in [nan_now_df_daily_cd, nan_now_df_latest_cd, nan_now_df_vaccination, nan_now_df_vaccination_md]:
    for idx, value in enumerate(item):
        print(value)

Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia
Namibia


In [72]:
# Check if there are any strings which contains the unwanted words
count = 0
change_name = {}
for item in [nan_now_df_daily_cd, nan_now_df_latest_cd, nan_now_df_vaccination, nan_now_df_vaccination_md]:
    for idx, value in enumerate(item):       
        try:
            if ('saint' in value.lower()) or ('sint' in value.lower()) or ('kosovo' in value.lower()):
                if count == 0:
                    now_df_daily_cd = now_df_daily_cd.drop(idx)
                elif count == 1:
                    now_df_latest_cd = now_df_latest_cd.drop(idx)
                elif count == 2:
                    now_df_vaccination = now_df_vaccination.drop(idx)
                else:
                    now_df_vaccination_md = now_df_vaccination_md.drop(idx)

            elif ('palestinian' in value.lower()) or ('palestine' in value.lower()):
                change_name[value] = 'State of Palestinian and Jerusalem'
                
        except Exception as e:
            print(e)
            pass
            
    count = count + 1

In [ ]:
now_df_daily_cd['country'] = now_df_daily_cd['country'].apply(lambda x: change_name.get(x) if x 
                                                              in change_name.keys() else x)
now_df_latest_cd['country'] = now_df_latest_cd['country'].apply(lambda x: change_name.get(x) if x 
                                                                in change_name.keys() else x)
now_df_vaccination['country'] = now_df_vaccination['country'].apply(lambda x: change_name.get(x) if x 
                                                                    in change_name.keys() else x)
now_df_vaccination_md['country'] = now_df_vaccination_md['country'].apply(lambda x: change_name.get(x) if x 
                                                                          in change_name.keys() else x)

In [66]:
def get_keys_from_values(dct, value):
    return [key for key, val in dct.items() if value == val]

dct1 = dict(zip(nan_now_df_daily_cd.index, nan_now_df_daily_cd.values)) # nan_now_df_daily_cd
dct2 = dict(zip(nan_now_df_latest_cd.index, nan_now_df_latest_cd.values)) # nan_now_df_latest_cd
dct3 = dict(zip(nan_now_df_vaccination.index, nan_now_df_vaccination.values)) # nan_now_df_vaccination
dct4 = dict(zip(nan_now_df_vaccination_md.index, nan_now_df_vaccination_md.values)) # nan_now_df_vaccination_md

# Find and delete None values in nan_now_df_daily_cd
for keys, values in dct1.items():
    if keys == None:
        # If both country name and country code is null, then check if there is ISO3 column
        if values == None:
            
            # If there is, then we can try to get the country name and code based on the ISO3 info
            try:
#                 now_df_latest_cd['iso3'] = 
            
            # Drop if no info
            except:
                now_df_latest_cd = now_df_latest_cd.drop(keys)
            
        else: 
            country_code_mapping_new
    

In [ ]:
# Find and delete None values in nan_now_df_latest_cd
if None in dct2.values():
    keys = get_keys_from_values(dct2, None)
    now_df_latest_cd = now_df_latest_cd.drop(keys)
    nan_now_df_latest_cd = nan_now_df_latest_cd.drop(keys)
    
# Find and delete None values in nan_now_df_vaccination
if None in dct3.values():
    keys = get_keys_from_values(dct3, None)
    now_df_vaccination = now_df_vaccination.drop(keys)
    nan_now_df_vaccination = nan_now_df_vaccination.drop(keys)
    
# Find and delete None values in nan_now_df_vaccination_md
if None in dct4.values():
    keys = get_keys_from_values(dct4, None)
    now_df_vaccination_md = now_df_vaccination_md.drop(keys)
    nan_now_df_vaccination_md = nan_now_df_vaccination_md.drop(keys)

## Apply function

In [ ]:
try:
    # Daily Case and Death
    # Pair of words with highest similarity for each country
    unique_countries = {country: most_similar(country) for country in now_df_daily_cd['country'].unique()}
    # Set the value
    now_df_daily_cd['country'] = now_df_daily_cd['country'].apply(lambda x: unique_countries.get(x))
    
    # Latest Case and Death
    now_df_latest_cd['country'] = now_df_latest_cd['country'].apply(most_similar)
    
    # Vaccination Data
    now_df_vaccination['country'] = now_df_vaccination['country'].apply(most_similar)
    
    # Vaccination Metadata
    now_df_vaccination_md['country'] = now_df_vaccination['country'].apply(most_similar)
    
    print("Successfully manipulated the Data.")
    print("================================")
    print()
    
except Exception as e:
    # Inconsistent Format Error
    inconsistent_format = logging.handlers.SMTPHandler(mailhost = ("smtp.example.com", 25),
                                                       fromaddr = "nabilfarras923@gmail.com", 
                                                       toaddrs = "nabilfarras923@gmail.com",
                                                       subject = u"COVID-19 Automation Failed Applying Function")
        
    logger = logging.getLogger()
    logger.addHandler(inconsistent_format)
    logger.exception('Unhandled Exception')
    
    print("Error found when manipulating the data.")

## Add Extra Feature(s)

For now, there will be only one feature we will add, which is the last update date in the now_df_latest_cd dataframe to give info when exactly the 'last 7 weeks' features are referred to.

In [ ]:
print("Adding Features .....")
print()

In [ ]:
# Daily Case and Death
# -

In [ ]:
# Latest Case and Death
# Fetch the max reported date for each country
max_date = now_df_daily_cd.groupby('country')[['date_reported']].max().reset_index(level = 0)
country_idx = max_date['country']
country_date = max_date['date_reported']

# Make it into dictionary
max_date = dict(zip(country_idx, country_date))
max_date['Global'] = max(max_date.values())

now_df_latest_cd['last_reported'] = now_df_latest_cd['country'].apply(lambda x: max_date.get(x))

In [ ]:
# Vaccination
# -

In [ ]:
# Vaccination Metadata
# -

# Check Format Data

Check if the yesterday's data format is different than today's data format, so we can adjust quickly.

In [ ]:
print("================================")
print("Checking Data Format .....")
print()

# Function to Check features
def check_features():
    count_features = 0
    for item in now_df_daily_cd.columns:
        if item not in yesterday_df_daily_cd.columns:
            count_features += 1
            
    for item in now_df_latest_cd.columns:
        if item not in yesterday_df_latest_cd.columns:
            count_features += 1
            
    for item in now_df_vaccination.columns:
        if item not in yesterday_df_vaccination.columns:
            count_features += 1
            
    for item in now_df_vaccination_md.columns:
        if item not in yesterday_df_vaccination_md.columns:
            count_features += 1

    if count_features != 0:
        print(f"Feature is not same. Adjustment needed in today's data features.")
        print()
        
        # Inconsistent Format Error
        inconsistent_format = logging.handlers.SMTPHandler(mailhost = ("smtp.example.com", 25),
                                                           fromaddr = "nabilfarras923@gmail.com", 
                                                           toaddrs = "nabilfarras923@gmail.com",
                                                           subject = u"COVID-19 Automation Inconsistent Data Format!")
        
        logger = logging.getLogger()
        logger.addHandler(inconsistent_format)
        logger.exception('Unhandled Exception')
        
    else:
        return "Pass. Data is ready to be imported."
        print()

In [ ]:
# # Check country rows, is yesterday's data a subset of today's data
# def check_subset():
#     count_subset = 0
#     for item in now_df_daily_cd['country'].unique():
#         if item not in yesterday_df_daily_cd['country'].unique():
#             count_subset += 1
            
#     for item in now_df_latest_cd['country'].unique():
#         if item not in yesterday_df_latest_cd['country'].unique():
#             count_subset += 1
            
#     for item in now_df_vaccination['country'].unique():
#         if item not in yesterday_df_vaccination['country'].unique():
#             count_subset += 1
            
#     for item in now_df_vaccination_md['country'].unique():
#         if item not in yesterday_df_vaccination_md['country'].unique():
#             count_subset += 1

#     if count_subset != 0:
#         print(f"Warning! Today's data is not a subset of yesterday's data.")
        
#         # Not a Subset Error
#         not_a_subset = logging.handlers.SMTPHandler(mailhost = ("smtp.example.com", 25),
#                                                     fromaddr = "nabilfarras923@gmail.com", 
#                                                     toaddrs = "nabilfarras923@gmail.com",
#                                                     subject = u"COVID-19 Automation Not a Subset!")
        
#         logger = logging.getLogger()
#         logger.addHandler(not_a_subset)
#         logger.exception('Unhandled Exception')
        
#     else:
#         return "Pass. Data is ready to be imported."
#         print()

In [ ]:
# df_dict = {'yesterday_df_daily_cd': 'now_df_daily_cd',
#           'yesterday_df_latest_cd': 'now_df_latest_cd',
#           'yesterday_df_vaccination': 'now_df_vaccination',
#           'yesterday_df_vaccination_md': 'now_df_vaccination_md'}

if cond == True: # We will check today's data compared to yesterday's data
    check_features()
#     check_subset()
    
else:
    print("There is no data recorded yet to compare with.")
    print()

print("Data Format have been checked.")
print("================================")
print()

# Feature Engineering

In [ ]:
# Create the cummulative cases and deaths for each country
today_cummulative_country = now_df_daily_cd.groupby(['country', 'who_region'])[['date_reported', 
                                                                                'cumulative_cases', 
                                                                                'cumulative_deaths']] \
                            .agg({'date_reported': 'max',
                                  'cumulative_cases': 'max', 
                                  'cumulative_deaths': 'max'}).reset_index()

today_cummulative_country

In [ ]:
# Create the cummulative cases and deaths for each date
today_cummulative = now_df_daily_cd.groupby('date_reported')[['cumulative_cases', 
                                                              'cumulative_deaths']] \
                    .agg({'cumulative_cases': 'sum', 
                          'cumulative_deaths': 'sum'}).reset_index()

today_cummulative

In [ ]:
# Create the cummulative cases and deaths for each region
today_cummulative_region = now_df_daily_cd.groupby('who_region')[['date_reported', 
                                                                   'cumulative_cases', 
                                                                   'cumulative_deaths']] \
                            .agg({'date_reported': 'max',
                                  'cumulative_cases': 'max', 
                                  'cumulative_deaths': 'max'}).reset_index()

today_cummulative_region

In [ ]:
# Create the cummulative vaccination info for each country
cumulative_vaccination_country = now_df_vaccination.copy()
cumulative_vaccination_country = cumulative_vaccination_country[['country_code', 'country', 'who_region', 'date_updated', 
                                                                 'total_vaccinations', 'persons_vaccinated_1plus_dose',
                                                                 'total_vaccinations_per100', 
                                                                 'persons_vaccinated_1plus_dose_per100',
                                                                 'persons_fully_vaccinated', 
                                                                 'persons_fully_vaccinated_per100',
                                                                 'vaccines_used', 'first_vaccine_date', 
                                                                 'number_vaccines_types_used',
                                                                 'persons_booster_add_dose', 
                                                                 'persons_booster_add_dose_per100']]

cumulative_vaccination_country['date_updated'] = dt.date.today()
cumulative_vaccination_country = cumulative_vaccination_country.reset_index(drop = True)
cumulative_vaccination_country

In [ ]:
# Create the cummulative vaccination info for each region
cumulative_vaccination_region = now_df_vaccination.copy()
cumulative_vaccination_region = cumulative_vaccination_region[['country_code', 'country', 'who_region', 'date_updated', 
                                                               'total_vaccinations', 'persons_vaccinated_1plus_dose',
                                                               'total_vaccinations_per100', 
                                                               'persons_vaccinated_1plus_dose_per100',
                                                               'persons_fully_vaccinated', 
                                                               'persons_fully_vaccinated_per100',
                                                               'vaccines_used', 'first_vaccine_date', 
                                                               'number_vaccines_types_used',
                                                               'persons_booster_add_dose', 
                                                               'persons_booster_add_dose_per100']]

cumulative_vaccination_region['date_updated'] = dt.date.today()

cumulative_vaccination_region = cumulative_vaccination_region.groupby('who_region') \
                                                             [['who_region', 'date_updated', 
                                                               'total_vaccinations', 'persons_vaccinated_1plus_dose',
                                                               'total_vaccinations_per100', 
                                                               'persons_vaccinated_1plus_dose_per100',
                                                               'persons_fully_vaccinated', 
                                                               'persons_fully_vaccinated_per100']] \
                                .agg({'date_updated': 'max',
                                      'total_vaccinations': 'sum', 
                                      'persons_vaccinated_1plus_dose': 'sum',
                                      'total_vaccinations_per100': 'mean', 
                                      'persons_vaccinated_1plus_dose_per100': 'mean',
                                      'persons_fully_vaccinated': 'sum', 
                                      'persons_fully_vaccinated_per100': 'mean'})

cumulative_vaccination_region

# Convert DataFrame into SQL Table in Local DB

## Delete Existing Table(s)

In [ ]:
print("================================")
print("Deleting Existing Tables .....")
print()

Base = declarative_base()
metadata = sql.MetaData()
insp = sql.inspect(sqlEngine)

def drop_table(tbl_name, engine = sqlEngine):
    
    # Since we are already connected to the local db, we can skip to the next step
    # which is dropping all the previous day' tables
    Base.metadata.drop_all(engine)
    metadata.reflect(bind = engine)
    table = metadata.tables[tbl_name]
    if table is not None:
        Base.metadata.drop_all(engine, [table], checkfirst = True)
        
for table in insp.get_table_names():
    drop_table(table)

print("No table remained in the local DB.")
print("================================")
print()

## Upload DataFrame to 'COVID-19' Local DB

In [ ]:
print("================================")
print("Uploading DataFrame .....")
print()

dfs = [now_df_daily_cd, now_df_latest_cd, now_df_vaccination, now_df_vaccination_md,
      today_cummulative_country, today_cummulative_region, cumulative_vaccination_country,
      cumulative_vaccination_region] # List of DataFrames

tbls = ['daily_case_death', 'latest_case_death', 'daily_vaccination', 'vaccination_md',
        'today_cummulative_country', 'today_cummulative_region', 'cumulative_vaccination_country',
        'cumulative_vaccination_region'] # List of table names

for i in range(len(dfs)):
    try:
        dfs[i].to_sql(tbls[i], sqlEngine, if_exists = "append", index = False)
        print("Successfully uploaded.")
        print()
        
    except Exception as e:
        print(f"Daily Case Death data error.")
        print(f"Type:", e)
        print()
        to_sql_error = logging.handlers.SMTPHandler(mailhost = ("smtp.example.com", 25),
                                                    fromaddr = "nabilfarras50@gmail.com", 
                                                    toaddrs = "nabilfarras923@gmail.com",
                                                    subject = u"COVID-19 Automation to SQL Error ({})!".format(value))

        logger = logging.getLogger()
        logger.addHandler(to_sql_error)
        logger.exception('Unhandled Exception')

print("All process ended.")
print("================================")